# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [ ]:
# Import this library
from nbacktest.backtest.engine import Backtest
from nbacktest.core.strategy import Strategy

# Import other libraries
import yfinance as yf

# Ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.broker.balance/2)/self.price["AAPL"])
        quantity2 = int((self.broker.balance/2)/self.price["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download df_ohlc Data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  130.173782  89.279305  130.726619  89.567592  127.982170   
2022-12-27  128.367203  87.410416  129.729560  88.971138  127.073957   
2022-12-28  124.428223  85.949112  129.354409  87.996937  124.260398   
2022-12-29  127.952568  88.424385  128.811438  88.836934  126.096612   
2022-12-30  128.268463  88.205704  128.288212  88.305111  125.800440   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.102251  129.245801  87.102251  63814900  17815000  
2022-12-27  87.017753  129.699945  88.782258  69007800  15470900  
2022-12-28  85.859647  128.011800  86.982968  85438400  17879600  
2022-12-29  86.475968  126.353282  86.515732  75703700  18280700  
2022-12-30  86.515744  126.767912  86.848764  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc, universe=universe, strategy=TestStrategy, price_column="Close", cash=100_000)

result = bt.run()

Strategy started!
Bought 1236 AAPL and 944 GOOG
Strategy finished!


In [5]:
result.head()

Price           Close                  High                   Low             \
Ticker           AAPL       GOOG       AAPL       GOOG       AAPL       GOOG   
Date                                                                           
2018-01-02  40.426823  52.935345  40.436212  53.031772  39.722768  51.952687   
2018-01-03  40.419777  53.804180  40.964248  53.993553  40.356415  52.846373   
2018-01-04  40.607529  53.999016  40.710791  54.355399  40.384579  53.879825   
2018-01-05  41.069862  54.785843  41.156695  54.886248  40.612227  54.277364   
2018-01-08  40.917324  55.019951  41.213026  55.235171  40.818753  54.755524   

Price            Open                Volume           ITERATION         EQUITY  
Ticker           AAPL       GOOG       AAPL      GOOG                           
Date                                                                            
2018-01-02  39.933986  52.107267  102223600  24752000         0  100000.000000  
2018-01-03  40.490183  52.901048  118071600  28604000         1  100811.472122  
2018-01-04  40.492532  54.078545   89738400  20092000         2  101227.458176  
2018-01-05  40.703754  54.376775   94640000  25582000         3  102541.667374  
2018-01-08  40.917324  54.785842   82271200  20952000         4  102574.127960

## <b> Backtest Statistics </b>

In [6]:
print("---------------------------")
print("Final balance: %.2f" % bt.broker.balance)
print("Final equity: %.2f" % bt.broker.equity)
print("---------------------------")

bt.broker.df_orderbook

---------------------------
Final balance: 61.48
Final equity: 241867.49
---------------------------


,ID,ITERATION,ACTION,TICKER,QUANTITY,PRICE,COMMISSION,SLIPPAGE,TOTAL,STATUS
0,372e9b29-9c4c-4d34-b442-23952bc069d6,0,buy,AAPL,1236,40.426823,0,0,-49967.552811,filled
1,fcebd9ce-f651-47d5-85b0-da7fb3776d4d,0,buy,GOOG,944,52.935345,0,0,-49970.965393,filled


In [7]:
bt.broker.df_positions

,QUANTITY,TOTAL_INVESTED,VALUE
TICKER,,,
AAPL,1236,-49967.552811,158539.820435
GOOG,944,-49970.965393,83266.184326


In [8]:
bt.broker.df_tradebook

,ID,STATUS,DESCRIPTION,PNL,CREATED_AT_ITERATION,CLOSED_AT_ITERATION,STOP_LOSS,TAKE_PROFIT,MAX_AGE,REASON_CLOSED
